In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchvision import transforms, datasets

In [0]:
apply_transform = transforms.Compose([
transforms.Resize(32)
,transforms.ToTensor()
])

BatchSize = 256

trainset = datasets.MNIST(root = './MNIST', train = True, download = True, transform = apply_transform)
trainLoader = torch.utils.data.DataLoader(trainset, batch_size = BatchSize, shuffle = True, num_workers = 10)

testset = datasets.MNIST(root = './MNIST', train = False, download = True, transform = apply_transform)
testLoader = torch.utils.data.DataLoader(testset, batch_size = BatchSize, shuffle = True, num_workers = 10)

Extracting ./MNIST/MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/MNIST/raw


Extracting ./MNIST/MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/MNIST/raw



Extracting ./MNIST/MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/MNIST/raw


Extracting ./MNIST/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [0]:
print(len(trainLoader.dataset))
print(len(testLoader.dataset))

60000
10000


In [0]:
class LeNet(nn.Module):
  def __init__(self):
    super(LeNet, self).__init__()
    self.conv1 = nn.Conv2d(1, 6, kernel_size = 5, bias = True)
    self.maxpool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.conv2 = nn.Conv2d(6, 16, kernel_size = 5, bias =True)
    self.maxpool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.fc1 = nn.Linear(400, 120, bias = True)
    self.fc2 = nn.Linear(120, 84, bias = True)
    self.fc3 = nn.Linear(84, 10, bias = True)
  
  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.maxpool1(x)
    x = F.relu(self.conv2(x))
    x = self.maxpool2(x)
    x = x.view(-1, 400)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    return F.log_softmax(x, dim = 1)
  
net = LeNet()
print(net)


LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [0]:
use_gpu = torch.cuda.is_available()
if use_gpu:
  print("GPU is available")
  net = net.cuda()

GPU is available


In [0]:
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
num_epochs = 50
train_loss = []
train_acc = []

for epoch in range(num_epochs):
  running_loss = 0.0
  running_corr = 0.0
  
  for i, data in enumerate(trainLoader):
    inputs, labels = data
    if use_gpu:
      inputs, labels = inputs.cuda(), labels.cuda()
    net.zero_grad()
    output = net(inputs)
    loss = criterion(output, labels)
    running_loss += loss
    preds = torch.argmax(output, dim = 1)
    running_corr = torch.sum(preds == labels) 

  totalLoss = running_loss / (i + 1)
  totalLoss.backward()
  
  for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
  
  epoch_loss = running_loss.item()/(i+1)
  epoch_acc = running_corr.item()/60000

  train_loss.append(epoch_loss)
  train_acc.append(epoch_acc)

  print('Epoch {:.0f}/{:.0f} : Training loss: {:.4f} | Training Accuracy: {:.4f}'.format(epoch+1,num_epochs,epoch_loss,epoch_acc*100))


Epoch 1/100 : Training loss: 2.3023 | Training Accuracy: 0.0133
Epoch 2/100 : Training loss: 2.3022 | Training Accuracy: 0.0150
Epoch 3/100 : Training loss: 2.3022 | Training Accuracy: 0.0067
Epoch 4/100 : Training loss: 2.3022 | Training Accuracy: 0.0117
Epoch 5/100 : Training loss: 2.3022 | Training Accuracy: 0.0067
Epoch 6/100 : Training loss: 2.3022 | Training Accuracy: 0.0100
Epoch 7/100 : Training loss: 2.3022 | Training Accuracy: 0.0067
Epoch 8/100 : Training loss: 2.3021 | Training Accuracy: 0.0133
Epoch 9/100 : Training loss: 2.3021 | Training Accuracy: 0.0117
Epoch 10/100 : Training loss: 2.3021 | Training Accuracy: 0.0067
Epoch 11/100 : Training loss: 2.3021 | Training Accuracy: 0.0133
Epoch 12/100 : Training loss: 2.3021 | Training Accuracy: 0.0050
Epoch 13/100 : Training loss: 2.3021 | Training Accuracy: 0.0067
Epoch 14/100 : Training loss: 2.3020 | Training Accuracy: 0.0083
Epoch 15/100 : Training loss: 2.3020 | Training Accuracy: 0.0150
Epoch 16/100 : Training loss: 2.30